In [ ]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

Mounted at /content/drive


# MAIN CODE

In [2]:
# set folder tempat kerja (current working directory)
import os

# cwd = '/content/drive/MyDrive/Monitoring Berita'
cwd = '/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita'
os.chdir(cwd)

In [8]:
# --- Sel 1: Import & Logging Config ---

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import logging
from urllib.parse import urljoin
from datetime import datetime
from requests.exceptions import RequestException, HTTPError, Timeout

# Konfigurasi logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S"
)

logging.info("Inisialisasi selesai.")


07:31:04 | INFO | Inisialisasi selesai.


In [9]:
# --- Sel 2: Parameter (mudah diubah) ---

import json

with open("config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

# Kata kunci topik untuk analisis relevansi judul
topic_keywords = config["keywords"]

# Daftar tanggal (YYYY-MM-DD). Akan di-convert ke DD-MM-YYYY untuk pencocokan di halaman.
dates = config["search_date"]

# Maksimum halaman per tanggal (akan berhenti lebih awal jika halaman kosong)
max_pages_per_date = config["max_page_length"]


In [10]:
# --- Sel 3: Kelas Scraper ---

class CnbcIndonesiaScraper:
    """
    Scraper indeks artikel CNBC Indonesia dengan analisis relevansi judul.
    Basis indeks: https://www.cnbcindonesia.com/indeks?tipe=artikel&page={page}
    """
    BASE_INDEX = "https://www.cnbcindonesia.com/indeks?tipe=artikel&page={page}"
    BASE_URL = "https://www.cnbcindonesia.com/"

    def __init__(self, timeout=10):
        self.timeout = timeout
        self.session = requests.Session()
        # User-Agent wajar
        self.session.headers.update({
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/122.0.0.0 Safari/537.36"
            ),
            "Accept-Language": "id,en;q=0.9"
        })

    # ------ Utilitas HTTP dengan retry (exponential backoff) ------
    def _get(self, url, max_retries=3):
        delay = 1
        for attempt in range(1, max_retries + 1):
            try:
                resp = self.session.get(url, timeout=self.timeout)
                resp.raise_for_status()
                return resp
            except (Timeout, HTTPError) as e:
                # Retry hanya untuk 5xx/timeout
                status = getattr(e.response, "status_code", None)
                if isinstance(e, Timeout) or (status and 500 <= status < 600):
                    logging.warning(f"Percobaan {attempt} gagal (akan retry): {e}")
                    if attempt < max_retries:
                        time.sleep(delay)
                        delay *= 2  # backoff
                        continue
                logging.error(f"Gagal GET {url}: {e}")
                raise
            except RequestException as e:
                logging.error(f"Kesalahan jaringan saat GET {url}: {e}")
                raise

    # ------ Bangun URL indeks per halaman ------
    def _build_index_url(self, page: int) -> str:
        return self.BASE_INDEX.format(page=page)

    # ------ Parsing list artikel dari halaman indeks ------
    def _parse_list_page(self, html) -> list:
        """
        Mengembalikan list elemen artikel (Tag) dari halaman indeks.
        Struktur situs dapat berubah; gunakan beberapa fallback selector.
        """
        soup = BeautifulSoup(html, "html.parser")

        # Beberapa pola umum listing artikel CNBC Indonesia
        candidates = []

        # 1) UL/OL list item
        candidates.extend(soup.select("ul li.article, ol li.article"))

        # 2) Card / item grid
        candidates.extend(soup.select("div.list, div.box, article"))

        # 3) Fallback: anchor ke artikel domain cnbcindonesia di dalam container daftar
        # (kita tidak langsung pakai semua <a> agar tidak kebanyakan noise)
        if not candidates:
            container = soup.select_one("div#indeks, div.container, main")
            if container:
                candidates = container.find_all(["article", "li", "div"], recursive=True)

        # Filter kasar: pastikan masing-masing block punya link ke cnbcindonesia.com
        filtered = []
        for tag in candidates:
            a = tag.find("a", href=True)
            if not a:
                continue
            href = a.get("href", "")
            if "cnbcindonesia.com" in href:
                filtered.append(tag)

        return filtered

    # ------ Ambil tanggal artikel sebagai teks (berbagai kemungkinan format) ------
    def _extract_date_text(self, tag) -> str:
        """
        Coba ekstrak tanggal dari beberapa selector umum:
        - <time datetime="..."> atau <time>...</time>
        - span/ div class mengandung 'date' / 'box__date'
        - small tag yang sering berisi tanggal
        Kembalikan string mentah; normalisasi dilakukan terpisah.
        """
        # time tag
        t = tag.find("time")
        if t:
            # prefer attribute datetime
            dt = t.get("datetime")
            if dt:
                return dt.strip()
            if t.text:
                return t.get_text(strip=True)

        # common classes
        for cls in ["date", "box__date", "box_date", "article__date", "media__date", "list__date"]:
            node = tag.find(class_=re.compile(cls, re.I))
            if node and node.get_text(strip=True):
                return node.get_text(strip=True)

        # small tag
        sm = tag.find("small")
        if sm and sm.get_text(strip=True):
            return sm.get_text(strip=True)

        return ""

    # ------ Normalisasi berbagai kemungkinan tanggal ke ISO (YYYY-MM-DD) jika memungkinkan ------
    def _normalize_date(self, raw_text: str) -> str:
        """
        CNBC Indonesia sering menampilkan format seperti '16 September 2025 10:23'
        atau '16-09-2025', atau pakai time[datetime].
        Kita akan coba beberapa pola lazim. Jika gagal, kembalikan raw_text apa adanya.
        """
        text = raw_text.strip()

        # 1) ISO/Datetime-like '2025-09-16' atau '2025-09-16T10:23:00+07:00'
        # -> parse bagian awal tanggalnya saja
        m_iso = re.match(r"(\d{4})-(\d{2})-(\d{2})", text)
        if m_iso:
            try:
                return f"{m_iso.group(1)}-{m_iso.group(2)}-{m_iso.group(3)}"
            except Exception:
                pass

        # 2) dd-mm-yyyy
        m_dmy_dash = re.search(r"(\d{2})-(\d{2})-(\d{4})", text)
        if m_dmy_dash:
            d, m, y = m_dmy_dash.groups()
            return f"{y}-{m}-{d}"

        # 3) dd/mm/yyyy
        m_dmy_slash = re.search(r"(\d{2})/(\d{2})/(\d{4})", text)
        if m_dmy_slash:
            d, m, y = m_dmy_slash.groups()
            return f"{y}-{m}-{d}"

        # 4) '16 September 2025' (Bahasa Indonesia)
        bulan_map = {
            "januari": 1, "februari": 2, "maret": 3, "april": 4, "mei": 5, "juni": 6,
            "juli": 7, "agustus": 8, "september": 9, "oktober": 10, "november": 11, "desember": 12
        }
        m_bi = re.search(
            r"(?P<d>\d{1,2})\s+(?P<mon>[A-Za-z]+)\s+(?P<y>\d{4})",
            text, flags=re.I
        )
        if m_bi:
            d = int(m_bi.group("d"))
            mon = bulan_map.get(m_bi.group("mon").lower(), None)
            y = int(m_bi.group("y"))
            if mon:
                return f"{y:04d}-{mon:02d}-{d:02d}"

        # Jika gagal normalisasi, kembalikan teks mentah (nanti caller bisa pakai fallback)
        return text

    # ------ Ekstrak satu artikel ------
    def _extract_article_data(self, tag, fallback_date_iso: str, topic_keywords: list) -> dict | None:
        try:
            a = tag.find("a", href=True)
            if not a:
                return None

            title = a.get("title") or a.get_text(strip=True)
            title = (title or "").strip()
            if not title:
                return None

            href = a["href"]
            url_abs = urljoin(self.BASE_URL, href)

            # Tanggal
            raw_date = self._extract_date_text(tag)
            normalized = self._normalize_date(raw_date) if raw_date else ""
            tanggal_berita = normalized if re.match(r"^\d{4}-\d{2}-\d{2}$", normalized) else fallback_date_iso

            # Penulis (CNBC kadang tidak menampilkan di listing; default Tidak Diketahui)
            penulis = "Tidak Diketahui"
            # Coba cari elemen umum yang kadang berisi penulis
            for cls in ["author", "penulis", "media__author", "list__author", "contributor"]:
                node = tag.find(class_=re.compile(cls, re.I))
                if node and node.get_text(strip=True):
                    penulis = node.get_text(strip=True)
                    break

            # Analisis relevansi
            title_norm = re.sub(r"[^\w\s]", " ", title).lower()
            found = [kw for kw in topic_keywords if re.search(rf"\b{re.escape(kw.lower())}\b", title_norm)]
            relevan = len(found) > 0

            return {
                "judul_berita": title,
                "tanggal_berita": tanggal_berita,
                "penulis_berita": penulis,
                "url_berita": url_abs,
                "relevan": relevan,
                "keywords_found": ", ".join(found)
            }
        except Exception as e:
            logging.warning(f"Gagal ekstrak artikel: {e}")
            return None

    # ------ Scrape satu tanggal ------
    def scrape_date(self, date_str: str, max_pages: int, topic_keywords: list) -> pd.DataFrame:
        """
        date_str: 'YYYY-MM-DD' (akan dipakai sebagai fallback & filter)
        Catatan: URL indeks CNBC tidak mengandung parameter tanggal, sehingga
        kita melakukan iterasi halaman dan memfilter berdasarkan tanggal yang terdeteksi.
        Jika elemen tanggal tidak tersedia di listing, fallback pakai date_str.
        """
        # Siapkan representasi target date untuk filter (DD-MM-YYYY) agar bisa cocok dengan tampilan situs jika diperlukan
        target_iso = date_str
        target_disp = datetime.strptime(date_str, "%Y-%m-%d").strftime("%d-%m-%Y")

        all_rows = []
        logging.info(f"Memproses tanggal: {date_str}")

        for page in range(1, max_pages + 1):
            url = self._build_index_url(page=page)
            try:
                resp = self._get(url)
            except Exception:
                logging.error(f"Stop iterasi (gagal mengambil halaman): {url}")
                break

            tags = self._parse_list_page(resp.text)
            logging.info(f"[{date_str}] Page {page} | URL: {url} | Artikel ditemukan (kandidat): {len(tags)}")

            if not tags:
                logging.info(f"[{date_str}] Halaman kosong atau pola listing tidak ditemukan. Stop iterasi.")
                break

            page_rows = []
            for tag in tags:
                row = self._extract_article_data(tag, fallback_date_iso=target_iso, topic_keywords=topic_keywords)
                if not row:
                    continue

                # Jika tanggal berhasil ternormalisasi, filter hanya tanggal target
                # (bila row['tanggal_berita'] berasal dari fallback, tetap akan sama dengan target)
                if re.match(r"^\d{4}-\d{2}-\d{2}$", row["tanggal_berita"]):
                    if row["tanggal_berita"] != target_iso:
                        # Abaikan artikel dari tanggal lain
                        continue
                else:
                    # Jika tidak ISO (jarang), cek apakah string punya bentuk DD-MM-YYYY yang cocok
                    if target_disp not in str(row["tanggal_berita"]):
                        continue

                page_rows.append(row)

            logging.info(f"[{date_str}] Page {page} | Artikel cocok tanggal {date_str}: {len(page_rows)}")
            all_rows.extend(page_rows)

            # Pengendalian beban
            time.sleep(1)

        if not all_rows:
            logging.warning(f"Tidak ada artikel untuk tanggal {date_str}.")

        return pd.DataFrame(all_rows, columns=[
            "judul_berita","tanggal_berita","penulis_berita","url_berita","relevan","keywords_found"
        ])

    # ------ Scrape banyak tanggal ------
    def scrape_many(self, dates: list, max_pages: int, topic_keywords: list) -> pd.DataFrame:
        frames = []
        for d in dates:
            try:
                df = self.scrape_date(d, max_pages, topic_keywords)
                frames.append(df)
            except Exception as e:
                logging.error(f"Gagal memproses tanggal {d}: {e}")
        if frames:
            out = pd.concat(frames, ignore_index=True)
            # Deduplicate by URL (jaga-jaga jika artikel muncul di banyak halaman)
            out = out.drop_duplicates(subset=["url_berita"]).reset_index(drop=True)
            return out
        return pd.DataFrame(columns=[
            "judul_berita","tanggal_berita","penulis_berita","url_berita","relevan","keywords_found"
        ])


In [11]:
# --- Sel 4: Eksekusi & Ringkasan ---

scraper = CnbcIndonesiaScraper(timeout=10)
df = scraper.scrape_many(dates=dates, max_pages=max_pages_per_date, topic_keywords=topic_keywords)

# Tampilkan DataFrame (hanya kolom yang diminta & urutan sesuai spesifikasi)
expected_cols = ["judul_berita","tanggal_berita","penulis_berita","url_berita","relevan","keywords_found"]
df = df.reindex(columns=expected_cols)

display(df)

# Ringkasan
total = len(df)
relevant = int(df["relevan"].sum()) if total > 0 else 0

print("\n--- Ringkasan ---")
print(f"Total artikel: {total}")
print(f"Artikel relevan: {relevant}")

if relevant > 0:
    examples = df[df["relevan"]].head(5)["judul_berita"].tolist()
    print("Contoh judul relevan:")
    for i, j in enumerate(examples, 1):
        print(f"{i}. {j}")
else:
    print("Tidak ada judul relevan untuk ditampilkan.")


07:31:21 | INFO | Memproses tanggal: 2025-09-24
07:31:21 | INFO | [2025-09-24] Page 1 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=1 | Artikel ditemukan (kandidat): 10
07:31:22 | INFO | [2025-09-24] Page 1 | Artikel cocok tanggal 2025-09-24: 10
07:31:23 | INFO | [2025-09-24] Page 2 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=2 | Artikel ditemukan (kandidat): 10
07:31:23 | INFO | [2025-09-24] Page 2 | Artikel cocok tanggal 2025-09-24: 10
07:31:24 | INFO | [2025-09-24] Page 3 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=3 | Artikel ditemukan (kandidat): 10
07:31:24 | INFO | [2025-09-24] Page 3 | Artikel cocok tanggal 2025-09-24: 10


,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Alasan Purbaya Incar Dana Pemda Rp 233 T yang ...,2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926070...,True,purbaya
1,"Laba Merdeka Battery (MBMA) US$ 5,8 Juta Semes...",2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/market/202509252...,False,
2,"TikTok Dijual Murah ke Amerika, Harganya Akhir...",2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/tech/20250926053...,False,
3,"Harga Emas Naik Tipis, Perak Melesat: Tapi Jan...",2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/research/2025092...,False,
4,InternasionalPanas! Israel Luncurkan Serangan ...,2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926062...,False,
5,"13 Makanan Favorit Warga RI yang Tinggi Garam,...",2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/lifestyle/202509...,False,
6,Kemenkeu: Sekolah Rakyat Sedot Anggaran Rp 788...,2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926065...,False,
7,"IHSG Anjlok 1%, Net Sell Tembus Rp 1 Triliun, ...",2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/market/202509252...,False,
8,Bapak Penemu HP Titip Pesan Menusuk Buat Warga...,2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/tech/20250926055...,False,
9,Harga Batu Bara Ikut Terpanggang Duel Sengit A...,2025-09-24,Tidak Diketahui,https://www.cnbcindonesia.com/research/2025092...,False,



--- Ringkasan ---
Total artikel: 30
Artikel relevan: 1
Contoh judul relevan:
1. Alasan Purbaya Incar Dana Pemda Rp 233 T yang Nganggur di BankNews4 menit yang lalu


In [12]:
df = df[['judul_berita', 'tanggal_berita', 'penulis_berita', 'url_berita', 'keywords_found']]
df.to_excel(cwd + "/daftar_berita/cnbc.xlsx", index=False)